# Unity Catalog Store Example

This notebook demonstrates how to use the `UnityCatalogStore` with LangGraph for persistent key-value storage.

## Setup and Configuration

In [ ]:
# Create widgets for configuration
dbutils.widgets.text("catalog", "", "Catalog Name")
dbutils.widgets.text("schema", "", "Schema Name")
dbutils.widgets.text("warehouse_id", "", "Warehouse ID")

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [ ]:
# Add the src directory to Python path for custom code imports
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [ ]:
# Enable MLflow autologging for LangChain
import mlflow
mlflow.langchain.autolog()
print("✓ MLflow LangChain autologging enabled")

In [ ]:
# Enable nested async support for Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()
print("✓ Nested asyncio support enabled")

In [ ]:
import os
from databricks.sdk import WorkspaceClient
from langgraph_unity_catalog_checkpoint.store import UnityCatalogStore

# Initialize Databricks WorkspaceClient
workspace_client: WorkspaceClient = WorkspaceClient()

# Configuration for Unity Catalog - prefer environment variables over widgets
catalog: str = os.getenv("UC_CATALOG") or dbutils.widgets.get("catalog")
schema: str = os.getenv("UC_SCHEMA") or dbutils.widgets.get("schema")
warehouse_id: str | None = os.getenv("DATABRICKS_WAREHOUSE_ID") or dbutils.widgets.get("warehouse_id") or None

print(f"Using catalog: {catalog}")
print(f"Using schema: {schema}")
print(f"Using warehouse_id: {warehouse_id}")

## Create Catalog and Schema

In [ ]:
import os
from databricks.sdk import WorkspaceClient
from langgraph.store.base import BaseStore
from langgraph_unity_catalog_checkpoint.store import UnityCatalogStore

# Initialize Databricks WorkspaceClient
workspace_client: WorkspaceClient = WorkspaceClient()

# Configuration for Unity Catalog - prefer environment variables over widgets
catalog: str = os.getenv("UC_CATALOG") or dbutils.widgets.get("catalog")
schema: str = os.getenv("UC_SCHEMA") or dbutils.widgets.get("schema")
warehouse_id: str | None = os.getenv("DATABRICKS_WAREHOUSE_ID") or dbutils.widgets.get("warehouse_id") or None

print(f"Using catalog: {catalog}")
print(f"Using schema: {schema}")
print(f"Using warehouse_id: {warehouse_id}")

## Part 1: Basic Store Operations

Demonstrate basic key-value operations with the Unity Catalog Store.

In [ ]:
# Create the store (uses default table name: "store")
store: UnityCatalogStore = UnityCatalogStore(
    workspace_client=workspace_client,
    catalog=catalog,
    schema=schema,
    warehouse_id=warehouse_id,
)

print(f"✓ Store created: {store.full_table_name}")

In [ ]:
# Define user_id for examples and memory namespacing
# This will be used throughout the notebook to demonstrate user-specific storage
user_id = "alice_123"

print(f"\u2713 Set user_id: {user_id}")

### Set Key-Value Pairs

In [ ]:
# Store some user preferences using BaseStore.put()
print("Setting user preferences...")

# Store individual items
store.put(("user", "alice", "preferences"), "theme", {"value": "dark"})
store.put(("user", "alice", "preferences"), "language", {"value": "python"})
store.put(("user", "bob", "preferences"), "theme", {"value": "light"})

print("✓ Stored 3 preferences using put()")

# Initialize Unity Catalog Store with BaseStore type hint
store: BaseStore = UnityCatalogStore(
    workspace_client=workspace_client,
    catalog=catalog,
    schema=schema,
    warehouse_id=warehouse_id,
)

print(f"\u2713 Unity Catalog Store initialized")
print(f"  Full table name: {store.full_table}")

In [ ]:
# Get values using BaseStore.get()
print("\nGetting values...")

alice_theme = store.get(("user", "alice", "preferences"), "theme")
alice_lang = store.get(("user", "alice", "preferences"), "language")
charlie_theme = store.get(("user", "charlie", "preferences"), "theme")  # Non-existent

print(f"Alice's theme: {alice_theme}")
print(f"Alice's language: {alice_lang}")
print(f"Charlie's theme: {charlie_theme}")  # Returns None

### List Keys with Prefix

In [ ]:
# Show all stored memories for this user in Unity Catalog
print(f"\nStored memories for user '{user_id}' in Unity Catalog:")
# Search using the same namespace as the memory tools: (user_id, "memories")
memory_items = store.search((user_id, "memories"), limit=100)
for item in memory_items:
    print(f"  Key: {item.key}")
    print(f"    Value: {item.value}")
    print(f"    Created: {item.created_at}")
    print()

print(f"Total memories stored: {len(memory_items)}")

### Delete Keys

In [ ]:
# Show all stored memories for this user in Unity Catalog
print(f"\nStored memories for user '{user_id}' in Unity Catalog:")
# Search using the same namespace as the memory tools: (user_id, "memories")
memory_items = store.search((user_id, "memories"), limit=100)
for item in memory_items:
    print(f"  Key: {item.key}")
    print(f"    Value: {item.value}")
    print(f"    Created: {item.created_at}")
    print()

print(f"Total memories stored: {len(memory_items)}")

## Part 2: LangGraph Agent with LangMem Tools

Use `langchain.agents.create_agent` to build an agent with memory capabilities.

In [ ]:
# Import LangMem tools and create_agent function
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain.agents import create_agent

# Initialize LLM
from databricks_langchain import ChatDatabricks
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")

print("✓ Imported create_agent and LangMem tools")

In [ ]:
# Create memory tools with user-specific namespace
# The namespace (user_id, "memories") ensures each user has isolated memory storage
namespace: tuple[str, ...] = ("memories", "{user_id}")
memory_tools = [
    create_manage_memory_tool(namespace=namespace),
    create_search_memory_tool(namespace=namespace),
]

print(f"\u2713 Created LangMem tools for user '{user_id}':")
for tool in memory_tools:
    print(f"  - {tool.name}: {tool.description}")

In [ ]:
# Create agent with memory tools and Unity Catalog store
agent = create_agent(
    model=llm,
    tools=memory_tools,
    store=store,  # Unity Catalog store for persistent memory
)

print("✓ Agent created with LangMem tools and Unity Catalog storage")

### Test the Agent - Store Memories

In [ ]:
# Configure the agent with user context
# Pass user_id in config to namespace memories to the specific user
config = {
    "configurable": {
        "user_id": user_id  # "alice_123"
    }
}

print(f"\u2713 Configured agent with user context: user_id = '{user_id}'")

In [ ]:
# First interaction - agent stores information
print("First interaction - storing memories:")
result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "My name is Alice and I love Python programming. I'm interested in AI and data science. Remember this!"
        }
    ]
}, config=config)  # Pass user context

print(f"\nAgent: {result['messages'][-1].content}")

### Test the Agent - Recall Memories

In [ ]:
# Second interaction - agent recalls stored information
print("\nSecond interaction - recalling memories:")
result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "What do you remember about me?"
        }
    ]
}, config=config)  # Pass user context

print(f"\nAgent: {result['messages'][-1].content}")

### View Stored Memories in Unity Catalog

In [ ]:
# Show all stored memories for this user in Unity Catalog
print(f"\nStored memories for user '{user_id}' in Unity Catalog:")
# Search using the same namespace as the memory tools: (user_id, "memories")
memory_items = store.search((user_id, "memories"), limit=100)
for item in memory_items:
    print(f"  Key: {item.key}")
    print(f"    Value: {item.value}")
    print(f"    Created: {item.created_at}")
    print()

print(f"Total memories stored: {len(memory_items)}")

## Summary

This notebook demonstrated:

### 1. Unity Catalog Store with BaseStore Interface
The Unity Catalog Store implements the [LangGraph BaseStore interface](https://github.com/langchain-ai/langgraph/blob/main/libs/checkpoint/langgraph/store/base/__init__.py):

```python
from langgraph.store.base import BaseStore
from langgraph_unity_catalog_checkpoint.store import UnityCatalogStore

store: BaseStore = UnityCatalogStore(
    workspace_client=workspace_client,
    catalog=catalog,
    schema=schema,
    warehouse_id=warehouse_id,
)
```

**BaseStore Methods Used:**
- `put(namespace, key, value)`: Store a single value
- `get(namespace, key)`: Retrieve a single value
- `search(namespace_prefix, ...)`: Search items in a namespace
- `delete(namespace, key)`: Delete an item
- `batch(ops)`: Execute multiple operations in batch

### 2. LangMem Integration with Unity Catalog
Using the official [LangMem library](https://langchain-ai.github.io/langmem/):

```python
from langchain.agents import create_agent
from langmem import create_manage_memory_tool, create_search_memory_tool

# Define user_id for namespace isolation
user_id = "alice_123"

# Create memory tools with user-specific namespace
memory_tools = [
    create_manage_memory_tool(namespace=(user_id, "memories")),
    create_search_memory_tool(namespace=(user_id, "memories")),
]

agent = create_agent(
    model=llm,
    tools=memory_tools,
    store=store,  # Unity Catalog store
)

# Pass user context when invoking
config = {"configurable": {"langgraph_user_id": user_id}}
result = agent.invoke({"messages": [...]}, config=config)
```

### Key Features

**LangMem Tools:**
- `create_manage_memory_tool` - Store, update, or delete memories
- `create_search_memory_tool` - Search memories using semantic similarity

**Unity Catalog Backend:**
- Enterprise-grade persistent storage
- Cross-session memory retention
- User-specific memory namespaces with isolation
- Hierarchical organization (user → memory type → specific memories)
- Semantic search capabilities (when configured with embeddings)

**Agent Capabilities:**
- Automatically stores important information from conversations
- Searches and recalls relevant memories when needed
- Maintains consistent context across sessions
- Personalizes responses based on stored knowledge
- Isolates memories per user for multi-tenant applications

### Learn More

- **LangMem:** [Official Documentation](https://langchain-ai.github.io/langmem/)
- **Memory Concepts:** [LangChain Memory Guide](https://docs.langchain.com/oss/python/concepts/memory)
- **LangGraph Persistence:** [Memory Storage Documentation](https://docs.langchain.com/oss/python/langgraph/persistence)
- **Unity Catalog:** [Databricks Documentation](https://docs.databricks.com/unity-catalog/index.html)